In [ ]:
# ! pip install --user librosa

In [ ]:
from pathlib import Path
# from scipy.io import wavfile
# import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
# import seaborn as sns
# import matplotlib.pyplot as plt
# from collections import Counter
import numpy as np
# import os
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.regularizers import l1, l2

## Mount Drive

In [ ]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [ ]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Colab Notebooks/Speech recognition')
  # intermediate_folder = Path('/content/gdrive/MyDrive/Temp/Speech recognition project')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Read data

In [ ]:
X_train = np.load(intermediate_folder / 'train_main_1_sec_wav2vec_emb_no_mixed.npy')
X_train.shape

(33566, 49, 768)

In [ ]:
X_val = np.load(intermediate_folder / 'val_main_1_sec_wav2vec_emb_no_mixed.npy')
X_val.shape

(4619, 49, 768)

In [ ]:
# X_test = np.load(intermediate_folder / 'test_main_1_sec_wav2vec_emb_no_mixed.npy')
# X_test.shape

In [ ]:
y_train_labels = pd.read_csv(intermediate_folder / 'train_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_val_labels = pd.read_csv(intermediate_folder / 'val_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_train_labels.shape, y_val_labels.shape
# y_train_labels.shape, y_val_labels.shape, y_test_labels.shape

((33566,), (4619,))

In [ ]:
le = LabelEncoder()
le.fit(y_train_labels)

y_train = le.transform(y_train_labels)
y_val = le.transform(y_val_labels)
# y_test = le.transform(y_test_labels)
y_train.shape, y_val.shape
# y_train.shape, y_val.shape, y_test.shape

((33566,), (4619,))

In [ ]:
pd.Series(y_train).value_counts().sort_index()

0     1667
1     1655
2     1696
3     1662
4     1647
5     1683
6     1723
7     1630
8     1668
9     1650
10    1672
11    1687
12    1708
13    1727
14    1715
15    1672
16    1693
17    1591
18    1686
19    1734
dtype: int64

In [ ]:
X_val.shape

(4619, 49, 768)

In [ ]:
assert np.mean(X_val[:, :, 0]) == np.mean(X_val.transpose(0, 2, 1)[:, 0, :])

In [ ]:
train_ind = np.random.permutation(len(y_train))
X_train = X_train[train_ind]
y_train = y_train[train_ind]
del train_ind
X_train.shape, y_train_labels.shape, y_train_labels.values

((33566, 49, 768),
 (33566,),
 array(['down', 'down', 'down', ..., 'zero', 'zero', 'zero'], dtype=object))

In [ ]:
del y_train_labels
del y_val_labels
# del y_test_labels

## Standardize

In [ ]:
MEAN = X_train.mean()
STD = np.std(X_train)
MEAN, STD

(-0.00032419193, 0.26497266)

In [ ]:
# X_train = (X_train - MEAN) / STD
# X_val = (X_val - MEAN) / STD
# # X_test = (X_test - MEAN) / STD
# X_train.shape, X_val.shape
# # X_train.shape, X_val.shape, X_test.shape

## Functions

In [ ]:
def train(model,
          patience_train,
          patience_val=None,
          learning_rate=.001,
          X_train=X_train,
          y_train=y_train,
          X_val=X_val,
          y_val=y_val,
          epochs=20,
          batch_size=128,
          checkpoint_file_name='best_model_after_wav.ckp',
          cont_train=False):
  print(f'{patience_train=}, {patience_val=}, {learning_rate=}, {epochs=}, {batch_size=}, {cont_train=}, {X_train.shape=}, {y_train.shape=}, {X_val.shape=}, {y_val.shape=}')
  model.summary()
  print(f'Input shape {model.input_shape}, output shape {model.output_shape}')
  callbacks = []
  if patience_train:
    callbacks.append(EarlyStopping(monitor='loss', patience=patience_train))
  if patience_val:
    callbacks.append(EarlyStopping(monitor='val_loss', patience=patience_val))

  callbacks.append(tf.keras.callbacks.ModelCheckpoint(
      filepath=(intermediate_folder / checkpoint_file_name),
      verbose=1,
      save_weights_only=True,
      save_freq='epoch',  # you can also use integer to specify after how many batches to save
      save_best_only=True  # if True, only the best model (lowest validation loss) is saved
      ))


  if not cont_train:
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  model.fit(X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks);

## CNN

In [ ]:
model = Sequential([Flatten(input_shape=(49, 768)),
                    # Conv2D(32, (2, 2), activation='relu', padding='same', input_shape=(32, 20, 1)),
                    # MaxPooling2D((2, 2)),
                    # Conv2D(32, (2, 2), activation='relu', padding='same'),
                    # MaxPooling2D((2, 2)),
                    # Conv2D(32, (2, 2), activation='relu', padding='same'),
                    # MaxPooling2D((2, 2)),
                    # Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 37632)             0         
                                                                 
 dense (Dense)               (None, 20)                752660    
                                                                 
Total params: 752,660
Trainable params: 752,660
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 49, 768), output shape (None, 20)
Epoch 1/1000
132/132 [==============================] - 10s 30ms/step - loss: 6.6625 - accuracy: 0.7893 - val_loss: 1.1534 - val_accuracy: 0.8974
Epoch 2/1000
132/132 [===============

In [ ]:
model = Sequential([Conv2D(16, (3, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((3, 1)),
                    Conv2D(16, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Conv2D(16, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 16)       64        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 768, 16)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 768, 16)       784       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 768, 16)       0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(8, (3, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((3, 1)),
                    Conv2D(8, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Conv2D(8, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 8)        32        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 768, 8)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 768, 8)        200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 768, 8)        0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (3, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((3, 1)),
                    Conv2D(4, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Conv2D(4, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        16        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 768, 4)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 768, 4)        52        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 768, 4)        0         
 2D)                                                             
                                           

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(2, (3, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((3, 1)),
                    Conv2D(2, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Conv2D(2, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 49, 768, 2)        8         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 768, 2)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 768, 2)        14        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 5, 768, 2)        0         
 2D)                                                             
                                          

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(2, (3, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((3, 1)),
                    Conv2D(2, (3, 1), activation='relu', padding='same'),
                    MaxPooling2D((3, 1)),
                    Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 2)        8         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 768, 2)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 768, 2)        14        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 768, 2)        0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(8, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 8)        400       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 8)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 20)                122900    
                                                                 
Total params: 123,300
Trainable params: 123,

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)                61460     
                                                                 
Total params: 61,660
Trainable params: 61,66

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(2, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 49, 768, 2)        100       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 2)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1536)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                30740     
                                                                 
Total params: 30,840
Trainable params: 30,

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(2, (15, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((15, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 2)        32        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 768, 2)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 20)                92180     
                                                                 
Total params: 92,212
Trainable params: 92,21

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (15, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((15, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 49, 768, 4)        64        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 768, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 9216)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                184340    
                                                                 
Total params: 184,404
Trainable params: 18

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (4, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((4, 1)),
                    Conv2D(4, (4, 1), activation='relu', padding='same'),
                    MaxPooling2D((4, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        20        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 768, 4)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 768, 4)        68        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 768, 4)        0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(8, (7, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((7, 1)),
                    Conv2D(8, (7, 1), activation='relu', padding='same'),
                    MaxPooling2D((7, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 49, 768, 8)        64        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 7, 768, 8)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 768, 8)         456       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 768, 8)        0         
 2D)                                                             
                                          

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (7, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((7, 1)),
                    Conv2D(4, (7, 1), activation='relu', padding='same'),
                    MaxPooling2D((7, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        32        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 768, 4)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 768, 4)         116       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 4)        0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(2, (7, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((7, 1)),
                    Conv2D(2, (7, 1), activation='relu', padding='same'),
                    MaxPooling2D((7, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 49, 768, 2)        16        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 7, 768, 2)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 768, 2)         30        
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 768, 2)        0         
 2D)                                                             
                                          

In [ ]:
model = Sequential([Conv2D(32, (7, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((7, 1)),
                    Conv2D(1, (7, 1), activation='relu', padding='same'),
                    MaxPooling2D((7, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 32)       256       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 768, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 768, 1)         225       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 1)        0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(64, (7, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((7, 1)),
                    Conv2D(1, (7, 1), activation='relu', padding='same'),
                    MaxPooling2D((7, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 64)       512       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 768, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 768, 1)         449       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 1)        0         
 2D)                                                             
                                            

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(1, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(1, (7, 1), activation='relu', padding='same'),
                    # MaxPooling2D((7, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 49, 768, 1)        50        
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 768, 1)        0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 768)               0         
                                                                 
 dense_2 (Dense)             (None, 20)                15380     
                                                                 
Total params: 15,430
Trainable params: 15,

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=15,
      patience_val=15,
      learning_rate=.001,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.001, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)                61460     
                                                                 
Total params: 61,660
Trainable params: 61,660

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=15,
      patience_val=15,
      learning_rate=.0001,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.0001, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                61460     
                                                                 
Total params: 61,660
Trainable params: 61,

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Dropout(.1, input_shape=(49, 768, 1)),
                    Conv2D(4, (49, 1), activation='relu', padding='same'),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 49, 768, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)     

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Dropout(.3, input_shape=(49, 768, 1)),
                    Conv2D(4, (49, 1), activation='relu', padding='same'),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    # Dropout(.2),
                    # Dense(32, activation='sigmoid'),
                    # Dropout(.5),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_1 (Dropout)         (None, 49, 768, 1)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 20)   

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.1),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dropout (Dropout)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)     

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.3),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dropout_1 (Dropout)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 20)   

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.5),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dropout (Dropout)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)     

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.4),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dropout_1 (Dropout)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 20)   

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.6),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.01,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.01, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dropout (Dropout)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)     

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.5),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=15,
      patience_val=15,
      learning_rate=.0001,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.0001, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 768, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dropout_1 (Dropout)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 20)   

KeyboardInterrupt: ignored

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.5),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=15,
      patience_val=15,
      learning_rate=.0001,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

patience_train=15, patience_val=15, learning_rate=0.0001, epochs=1000, batch_size=256, cont_train=False, X_train.shape=(33566, 49, 768), y_train.shape=(33566,), X_val.shape=(4619, 49, 768), y_val.shape=(4619,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 768, 4)        200       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 768, 4)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dropout (Dropout)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 20)     

In [ ]:
model = Sequential([Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
                    MaxPooling2D((49, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    # Conv2D(2, (3, 1), activation='relu', padding='same'),
                    # MaxPooling2D((3, 1)),
                    Flatten(),
                    Dropout(.5),
                    # Dense(32, activation='relu'),
                    # Dropout(.5),
                    # Dense(128, activation='relu'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])

train(model=model,
      patience_train=15,
      patience_val=15,
      learning_rate=.0001,
      epochs=1000,
      batch_size=256,
      # cont_train=True,
      )

## End